In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Pipeline untuk setiap model
log_reg_pipe = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi untuk Logistic Regression
    ('lr', LogisticRegression())
])

rf_pipe = Pipeline([
    ('rf', RandomForestClassifier())  # Random Forest tidak butuh scaling
])

knn_pipe = Pipeline([
    ('scaler', StandardScaler()),  # Normalisasi untuk KNN
    ('knn', KNeighborsClassifier())
])

In [ ]:
# Voting Classifier dengan Pipeline
voting_clf = VotingClassifier(estimators=[
    ('lr_pipe', log_reg_pipe),
    ('rf_pipe', rf_pipe),
    ('knn_pipe', knn_pipe)
], voting='hard')

In [ ]:
# Hyperparameter tuning untuk masing-masing pipeline dalam Voting Classifier
param_dist = {
    'lr_pipe_lr_C': np.logspace(-3, 2, 10),
    'rf_pipe_rf_n_estimators': [50, 100, 200],
    'rf_pipe_rf_max_depth': [None, 10, 20, 30],
    'knn_pipe_knn_n_neighbors': range(1, 20),
    'knn_pipe_knn_weights': ['uniform', 'distance']
}

In [ ]:
# RandomizedSearchCV untuk mencari kombinasi terbaik
random_search = RandomizedSearchCV(voting_clf, param_distributions=param_dist, n_iter=20, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

In [ ]:
# Evaluasi pada test set
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Hasil
print(f"Best Parameters: {random_search.best_params_}")
print(f"Test Accuracy: {accuracy:.4f}")